In [156]:
using LinearAlgebra, DynamicPolynomials, SparseArrays, DataFrames, PrettyTables
include("methods.jl");

## Identifiability up to rank ${n\choose 2} + 1$

Let 
$$
    V = \mathbb \{p^3 \mid p \in \mathbb P(S^2(\mathbb C^n)) \}
$$

Consider the specific instance of ${n\choose 2}$ quadratics
$$
    q_{ij} = (X_i + X_j)^2, \qquad (i, j \in \{1,\ldots, n\}, i < j)
$$
Let $m:={n\choose 2} + 1$ and $q = (q_1, \ldots, q_{m})$ be a vector whose entries are $q_1 := X_1^2 $ and the $q_{ij}$. We claim that 
1. the tangent spaces $T_{q_1} V, \ldots, T_{q_m} V$ are skew.
2. the contact locus
$$
\mathcal C([q_1], \ldots, [q_{m}]) = \{p \in \mathbb P(S^2(\mathbb C^n)) \mid T_pV \subseteq \sum_{k=1}^m T_{[q_k]}V \}
$$
has (projective) dimension zero locally around each of $[q_1],\ldots,[q_m]$. NB: Since our specific instance is symmetric under permutations of $\{2,\ldots, m\}$, it suffices to compute the dimension of $\mathcal C([q_1], \ldots, [q_{m}])$ at $X_1^2$, $(X_1 + X_2)^2$ and $(X_2 + X_3)^2$. 

We start by constructing the quadratics $q_1, \ldots, q_m$. All computations will be done in affine language. 

In [147]:
n = 5;
@polyvar X[1:n] # base variables on ℂⁿ
N = length(monomials(X, 6)) # 210 for n=5
@polyvar Y[1:N] # variables for the space S⁶(ℂⁿ)
M = length(monomials(X, 2)) # 15 for n=5
@polyvar Z[1:M] # variables for the space S²(ℂⁿ)

q = [(X[i] + X[j])^2 for i=1:n, j=1:n if i<j] 
push!(q, X[1]^2)

m=length(q) # should be binomial(n, 2) + 1 
display(q)

11-element Vector{Polynomial{true, Int64}}:
 X₁² + 2X₁X₂ + X₂²
 X₁² + 2X₁X₃ + X₃²
 X₂² + 2X₂X₃ + X₃²
 X₁² + 2X₁X₄ + X₄²
 X₂² + 2X₂X₄ + X₄²
 X₃² + 2X₃X₄ + X₄²
 X₁² + 2X₁X₅ + X₅²
 X₂² + 2X₂X₅ + X₅²
 X₃² + 2X₃X₅ + X₅²
 X₄² + 2X₄X₅ + X₅²
 X₁²

In [148]:
# The constructed set is stable under substitution: Take any variable X[i] where i is not 1 and substitue X[i] by X[1]
# Then we get a set which is structurally the same but in one variable less.
# To try it out, uncomment the following two lines of code. Formal proof is in the mathematical part. 

# q = unique([subs(g, X[n]=>X[1]) for g in q])
# m = length(q)

In [149]:
A = tangent_space(q, X)
(n_rows, n_cols) = size(A)

In [150]:
df_skewtangents = DataFrame(:expected_dimension => m*M, :actual_dimension => rank(Matrix(A)))
dict_skewtangents = Dict("1: Expected Dimension of Secant" => m*M, 
                         "2: Actual Dimension of Secant" =>rank(Matrix(A)),
                         "3: Ambient Space Dimension (S⁶(ℂⁿ))" => N, 
                         "4: Parameter Space Dimension (S²(ℂⁿ))" => M)
pretty_table(dict_skewtangents, noheader=true, alignment=:l, sortkeys=true)

┌───────────────────────────────────────┬─────┐
│ 1: Expected Dimension of Secant       │ 165 │
│ 2: Actual Dimension of Secant         │ 165 │
│ 3: Ambient Space Dimension (S⁶(ℂⁿ))   │ 210 │
│ 4: Parameter Space Dimension (S²(ℂⁿ)) │ 15  │
└───────────────────────────────────────┴─────┘


The above calculation shows that the tangent spaces $T_{[q_1]} V,\ldots, T_{[q_m]} V$ are skew. For the contact locus, we first construct linear equations for the subspace $\sum_{k=1}^m T_{[q_k]}V $.

In [151]:
Q=qr(Matrix(A)).Q
equations = (@view Q[:, n_cols+1:n_rows])';

Now, for each of $q_1 = (X_1 + X_2)^2$, $q_2 = (X_2 + X_3)^2$ and $q_{11} = X_1^2$, we check that the affine contact locus consists locally only of the line through $q_1, q_2$ or $q_{11}$, respectively. Observe that each of the linear equations $f$ for $\sum_{k=1}^m T_{q_k}V$ that we computed earlier gives ${n\choose 2}$ quadratic equations 
$$
    (\ast) \qquad f(p^2 X_iX_j) = 0, \qquad p\in S^2(\mathbb C ^n), i, j \in \{1,\ldots, n\}
$$
in $p$ which alltogether describe the containment $T_p V \subseteq \sum_{k=1}^m T_{q_k}V$. Deriving the equations $(\ast)$ by $p$ gives a system of linear equations $L_p$ in $p$ which describe a supervariety $W$ of $T_{[p]} \mathcal C([q_1],\ldots, [q_m])$. 
Thus we may upper-bound the dimension of $C([q_1],\ldots, [q_m])$ at any given point $p$ by the dimension of the kernel of $L_p$ minus one.

Instead of performing the procedure of $(\ast)$ for **all** 45 linear equations for $\sum_{k=1}^m T_{q_k}V$, note that we can decide to use less: In fact, we only use two. This might a priori give us a worse upper bound, but it will turn out that the 30 quadratic equations obtained via $(\ast)$ from two linear ones are already sufficient to upper bound the dimension by 1 at all points of interests, which is what we want.  


In [152]:
𝒟 = Dict([mon=>i for (mon, i) in zip(monomials(X, 6), 1:N)])
""" 
`x_coefficients(poly)`
Takes a polynomial `poly` in X, Z and returns coefficients only wrt to X (as polynomials in Z) 
"""
x_coefficients = function(poly)
    v = [Polynomial(Z[1]^0-1) for i=1:N] # array initialised with zero polynomials
    hess = (1/2).*differentiate(poly, Z, Val{2}())  # first remove the Z variables. We get them back later from the position in the matrix. 
    indices = vec([𝒟[a[1]] for a in monomials.(hess)])      # each entry in hess is a degree 6 monomial in X. We convert these to positions in the resulting vector 
    values  = vec([c[1] for c in coefficients.(hess)].*(Z*Z'))
    for (idx, i) in zip(indices, 1:M^2)
        v[idx] += values[i] 
    end
    return v
end;

In [154]:
# plug in polynomials of the kind f²XᵢXⱼ (where f is a quadratic with variable coefficients) into the equations:
p = Z⋅monomials(X, 2); # quadratic with unknown coefficients 

# compute the embedding from S²(ℂ)² to subspaces of S⁶(ℂ) that maps p² to the space spanned by p²XᵢXⱼ
# x_coefficients is used to represent the p²XᵢXⱼ w.r.t. the standard basis monomials(X, 6) of S⁶(ℂ) 
P = [x_coefficients(p^2*X[i]*X[j]) for i=1:n, j=1:n if i≤j];

quadratic_equations = [(@view equations[i, :])⋅poly for poly in P, i=1:2];

In [155]:
function ts_eq_at(p, variety_eq) 
    ts_eq = differentiate(variety_eq, Z) # computes one equation for the tangent space as a (gradient) vector whose entries are deg 1 polynomials
    return [f(Z=>coefficients(p, monomials(X, 2))) for f in ts_eq] # evaluates the gradient at specific point to obtain a column vector describing the linear equation for the tangent space at that point
end

function L(p)
    ts_eqs = [];
    for quad_eq in quadratic_equations
        push!(ts_eqs, ts_eq_at(p, quad_eq))
    end
    return hcat(ts_eqs...)
end

df_cl_dimensions = DataFrame("Point" => "Contact Locus Dimension", "X₁²" => M-rank(L(q[11])),  "(X₁ + X₂)²" => M-rank(L(q[1])), "(X₂ + X₃)²" => M-rank(L(q[2])))
pretty_table(df_cl_dimensions, nosubheader=true)


┌─────────────────────────┬─────┬────────────┬────────────┐
│                   Point │ X₁² │ (X₁ + X₂)² │ (X₂ + X₃)² │
├─────────────────────────┼─────┼────────────┼────────────┤
│ Contact Locus Dimension │   1 │          1 │          1 │
└─────────────────────────┴─────┴────────────┴────────────┘


The computation shows that locally around $X_1^2, (X_1 + X_2)^2$ and $(X_2 + X_3)^2$, the affine cone of the contact locus at $q_1,\ldots, q_m$ consists only of the lines through $X_1^2, (X_1 + X_2)^2$ and $(X_2 + X_3)^2$, respectively, which is exactly what we claimed. 